In [16]:
#read text with fitz
import fitz
doc = fitz.open(r"C:\Users\leoac\Work\Companies\Loomy (personal)\loomy\kb\old\provvedimenti_soggetti_a_pubblicita_2016 - Provvedimenti del Direttore soggetti a pubblicità legale_Ottobre 2016 - Provvedimenti del Direttore soggetti a pubblicità legale_Definizione.pdf")
text = ""
for page in doc:
    text += page.get_text()

for para in text.split("\n\n"):
    print(para)

API REST Gestionali
API per l'utilizzo dei servizi di gestionali per gestori e produttori.
Version 3.0
Paths
/gestori/me/dispositivi/
GET /gestori/me/dispositivi/
Summary
Elenco dei dispositivi del gestore.
Description
Consente di avere l'elenco dei dispositivi associati al gestore con una serie di filtri.
Assicurarsi che i valori relativi al content-type siano conformi a quanto previsto:
produces: - application/json
Parameters
Name
Located
in
Description
Required Schema
idDispositivo query Identificativo del dispositivo assegnato
dal gestore o produttore
No
⇄string
tipologia
query Filtro per tipologia di dispositivo
No
⇄string
stato
query Filtro per stato del dispositivo
No
⇄string
start
query Scostamento dal primo elemento
restituito
No
⇄string
perPage
query Numero di elementi per pagina (max
50)
No
⇄string
colsIdx
query Numeri di colonna da ordinare
No
⇄string
colsOrder
query Tipo di ordinamento in base alle
colonne specificate su colsIdx
No
⇄string
Responses
Code
Description
Schema

In [1]:
from pinecone import Pinecone
import os
import random

pinecone_api_key = os.getenv("PINECONE_API_KEY")
if not pinecone_api_key:
    raise ValueError("PINECONE_API_KEY environment variable not set")
pc = Pinecone(api_key=pinecone_api_key)

def query_index(index_name: str, namespace: str | None, metadata_filter: dict | None, query_embedding, top_k):
        if not pc.has_index(index_name):
            print(f"[retrieve] Index not found: {index_name}")
            return []
        index = pc.Index(name=index_name)
        print(f"[retrieve] Querying Pinecone index: {index_name} namespace: {namespace}")
        response = index.query(
            namespace=namespace,
            vector=query_embedding,
            top_k=top_k,
            include_metadata=True,
            include_values=False,
            filter=metadata_filter
        )
        return response.get("matches", [])

In [2]:
import cohere
cohere_api_key = os.getenv("COHERE_API_KEY")
co = cohere.ClientV2(cohere_api_key)

# generate a query_embedding vector of dim 1536 with cohere embedv4
# query = "Aliquota IVA applicabile alle cessioni di smoothies"
query = ""
query_embedding = co.embed(texts=[query], input_type="search_query", model="embed-v4.0", output_dimension=1536).embeddings.float_[0]
print(query_embedding)
# query_embedding = [random.random() for _ in range(1536)]
query_index("public", metadata_filter={"library": {"$eq": "public"}}, namespace=None, query_embedding=query_embedding, top_k=5)

[0.0071862745, -0.009160526, 0.003000862, -0.018873842, -0.024322776, 0.0021420626, -0.012714177, -0.002309874, 0.009555377, 0.024954535, -0.010581986, 0.10234518, -0.01610989, -0.012240358, -0.020058393, 0.018794872, 0.0043630954, 0.022585435, 0.025428355, 0.020137362, 0.024796596, -0.008607735, -0.017452382, -0.011055808, -0.04422323, 0.021321913, 0.045802627, 0.0090815555, 0.007897005, 0.0075021544, -0.058753718, -0.006396574, -0.013740788, 0.025744235, -0.051804356, 0.029060978, 0.052751996, -0.015004309, -0.0065150294, 0.027955396, 0.014372549, 0.025270415, -0.013030059, -0.017768262, -0.01603092, -0.003652365, 0.0068309093, -0.0030206046, -0.041380305, 0.030166559, -0.0021716764, 0.01571504, -0.023533074, -0.024796596, 0.07328421, -0.01761032, 0.0018163112, 0.012872118, 0.0057648136, 0.039327085, -0.009515891, 0.015004309, -0.04453911, 0.016425772, 0.0053699636, 0.034273002, -0.017373411, 0.016346801, 0.01579401, 0.0047184606, -0.014135639, 0.0020630925, -0.01761032, 0.026533937,

c:\Users\leoac\Work\Companies\Loomy (personal)\loomy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[retrieve] Querying Pinecone index: public namespace: None


[{'id': '_pdf-16e1b0c3-8a86-439c-8916-34bff8e4af4d',
  'metadata': {'chunk_text': '}',
               'doc_name': ' pdf',
               'library': 'public',
               'page': 27.0,
               'source': 'provvedimenti_soggetti_a_pubblicita',
               'storage_path': 'provvedimenti_soggetti_a_pubblicita/2016 - '
                               'Provvedimenti del Direttore soggetti a '
                               'pubblicità legale/Novembre 2016 - Provvedimenti '
                               'del Direttore soggetti a pubblicità '
                               'legale/Modificazioni degli allegati alle '
                               'specifiche tecniche approvate con provvedimento '
                               'del Direttore dell’Agenzia delle entrate del 30 '
                               'giugno 2016 relativo alla definizione delle '
                               'informazioni, delle regole tecniche, degli '
                               'strumenti e dei termi

In [8]:
# write query_embedding to a txt file
with open("query_embedding.txt", "w") as f:
    for value in query_embedding:
        f.write(f"{value}\n")

In [15]:
from pydantic import BaseModel
class RetrieveRequest(BaseModel):
    query: str
    index_name: str
    namespace: str | None = None  # Required when "private" is selected
    libraries: list[str]  # e.g. ["organization", "private", "global"]
    top_k: int = 5

In [18]:
def retrieve(retrieve_request: RetrieveRequest):
    """Retrieve similar documents from the Pinecone vector store."""
    print("[retrieve] Start retrieval process")

    requested_libraries = set(retrieve_request.libraries)
    print("Retrieve user: ", retrieve_request.namespace)
    print("Requested libraries: ", requested_libraries)
    print("Top K: ", retrieve_request.top_k)

In [19]:
query = "Prova"
index_name = "example_index"
namespace = "example_namespace"
libraries = ["organization", "private"]
top_k = 5

retrieve_request = {
    "query": query,
    "index_name": index_name,
    "namespace": namespace,
    "libraries": libraries,
    "top_k": top_k
}
retrieve(RetrieveRequest(**retrieve_request))

[retrieve] Start retrieval process
Retrieve user:  example_namespace
Requested libraries:  {'organization', 'private'}
Top K:  5


In [24]:
r = {
                "status": "success",
                "query": "Ciao",
                "results": ["retrieved_docs", "a", "b"],
                "total_results": 3,
            }
r.get("results", [])

['retrieved_docs', 'a', 'b']

In [26]:
import uuid

def generate_unique_id():
    return str(uuid.uuid4())
unique_id = generate_unique_id()
print("Generated Unique ID:", unique_id)

Generated Unique ID: 91d79db5-92db-4131-913a-dfbc7993c8e4


In [ ]:
from psycopg2 import pool
from dotenv import load_dotenv
import os

# Load environment var\iables from .env
load_dotenv()
conversation_id = "f5297ad9-1e04-4762-8de2-1c62431b35ea"
# Fetch variables
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")


class DBUtils:
    _connection_pool = None
    
    @classmethod
    def initialize_pool(cls, minconn=1, maxconn=10):
        """Initialize connection pool once at app startup"""
        if cls._connection_pool is None:
            cls._connection_pool = pool.SimpleConnectionPool(
                minconn, maxconn,
                user=DB_USER,
                password=DB_PASSWORD,
                host=DB_HOST,
                port=DB_PORT,
                dbname=DB_NAME
            )
    
    @classmethod
    def get_connection(cls):
        """Get a connection from the pool"""
        return cls._connection_pool.getconn()
    
    @classmethod
    def return_connection(cls, connection):
        """Return connection back to pool"""
        cls._connection_pool.putconn(connection)
    
    @classmethod
    def execute_query(cls, query, params=None):
        """Execute query and return results"""
        connection = cls.get_connection()
        try:
            with connection.cursor() as cursor:
                cursor.execute(query, params)
                # connection.commit() only needed for INSERT/UPDATE/DELETE
                return cursor.fetchall()
        finally:
            cls.return_connection(connection)

# Initialize once at app startup
DBUtils.initialize_pool()

# Use for each prompt
chat_history = DBUtils.execute_query(
    "SELECT * FROM messages WHERE conversation_id = %s ORDER BY created_at DESC LIMIT 5",
    (conversation_id,)
)

In [ ]:
messages = []
for msg in chat_history:
    if msg[6] != 'error':
        message_content = {
            "sender": msg[2],
            "content": msg[3],
            "metadata": msg[4]
            # "created_at": msg[5]
        }
        messages.append(message_content)
messages

[{'sender': 'bot',
  'content': 'Si è verificato un errore durante la generazione della risposta. Per favore riprova.',
  'metadata': None},
 {'sender': 'user', 'content': 'Dimmi cosa è un bilancio', 'metadata': None},
 {'sender': 'bot',
  'content': 'Si è verificato un errore durante la generazione della risposta. Per favore riprova.',
  'metadata': None},
 {'sender': 'user', 'content': 'Dimmi cosa è un bilancio', 'metadata': None}]

In [8]:
import uuid
from pinecone import Pinecone, ServerlessSpec
import cohere
import openai
import PyPDF2
import os
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

# initialize cohere
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
co = cohere.Client(COHERE_API_KEY)
embedding_model_name = "embed-v4.0"

# openai doesn't need to be initialized, but need to set api key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
def extract_text_from_pdf(pdf_path):
    """Extract text content from a PDF file"""
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
        return text.strip()
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return None

In [3]:
doc_text = extract_text_from_pdf(pdf_path="..\kb\Lecture_0.pdf")

In [5]:
response = co.embed(
    texts=[doc_text],
    model=embedding_model_name,
    input_type="search_document",
    embedding_types=["float"]
)

In [6]:
len(response.embeddings.float_[0]) # 1536

1536

In [ ]:
import cohere
import pinecone
import uuid

# 1. Initialize clients
co = cohere.Client("YOUR_COHERE_API_KEY")
pinecone.init(api_key="YOUR_PINECONE_API_KEY", environment="YOUR_ENV")  # e.g., us-west1-gcp

# 2. Ensure index exists (or create it)
INDEX_NAME = "example-index"
if INDEX_NAME not in pinecone.list_indexes():
    pinecone.create_index(
        name=INDEX_NAME,
        dimension=1024,        # must match your embedding size
        metric="cosine"
    )
index = pinecone.Index(INDEX_NAME)

# 3. Your text to embed (embedded previously via Cohere)
texts = [
    "First document text",
    "Second doc text",
    # ...
]

# 4. Generate embeddings with Cohere
response = co.embed(
    texts=texts,
    model="embed-english-v3.0",
    input_type="search_document",
    truncate="END"
)
embeddings = response.embeddings  # list of vectors length 1024 :contentReference[oaicite:1]{index=1}

# 5. Prepare records for upsert
vectors = []
for text, vec in zip(texts, embeddings):
    # Use consistent unique IDs so you can re-upsert (updates instead of duplicates)
    vid = str(uuid.uuid5(uuid.NAMESPACE_URL, text))
    vectors.append((vid, vec, {"text": text}))

# 6. Upsert (batching if many)
index.upsert(vectors=vectors, namespace="my-namespace")

print(f"Upserted {len(vectors)} vectors into {INDEX_NAME}")
